Creating a Classification Model
==
**Authors**: Bao Dinh, Wesley Smith, Dakoda Fisher, Mohammed Qurneh, Andreas Cedron

**Research Question:** How do personal and occupational factors affect income?

**Dataset:** *Census Income* dataset extracted by Barry Becker in 1994 and donated to the [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Census+Income)

**Problem Statement:** Using the *Census Income* dataset, we'll build a classification model to predict whether the income of a U.S. individual will be greater than or less than $50,000 per year.

# Importing Libararies
We'll begin by importing the necessary libraries for creating a classification model.

In [26]:
# libaries for data processing
import numpy as np
import pandas as pd

# libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# modules for encoding variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# modules for modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# modules for model evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

# Loading Dataset
As mentioned before, we'll use the *Census Income* dataset from the link above. We'll start by loading the dataset from the UC Irvine Machine Learning Repository.

In [3]:
# install library to connect to the repository
!pip install ucimlrepo
# import libraries
from ucimlrepo import fetch_ucirepo

In [4]:
# fetch dataset
adult = fetch_ucirepo(id=2)
# store features and targets as pandas dataframes
X = adult.data.features
y = adult.data.targets
# print database metadata
print(adult.metadata)
# concatenate features and targets
data = pd.concat([X, y], axis=1)

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

# Initial Data Exploration
It is a good idea to understand the overall structure of our data before we begin processing dataset.

In [5]:
df = data.copy()

# shape of dataset
print("Shape of dataframe is:", df.shape, "\n")
# summary of data
df.info()
# statistical details of data
df.describe().T

Shape of dataframe is: (48842, 15) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


,count,mean,std,min,25%,50%,75%,max
age,48842.0,38.643585,13.710510,17.0,28.0,37.0,48.0,90.0
fnlwgt,48842.0,189664.134597,105604.025423,12285.0,117550.5,178144.5,237642.0,1490400.0
education-num,48842.0,10.078089,2.570973,1.0,9.0,10.0,12.0,16.0
capital-gain,48842.0,1079.067626,7452.019058,0.0,0.0,0.0,0.0,99999.0
capital-loss,48842.0,87.502314,403.004552,0.0,0.0,0.0,0.0,4356.0
hours-per-week,48842.0,40.422382,12.391444,1.0,40.0,40.0,45.0,99.0


In [6]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


From the code output above, we can see our dataset has a total of **`15 variables`** and **`48842 observations`**. **`income`** is our target variable.

Our **6** numerical features are: **`age`**, **`fnlwgt`**, **`education-num`**, **`capital-gain`**, **`capital-loss`**, and **`hours-per-week`**

Our **9** categorical features are: **`workclass`**, **`education`**, **`marital-status`**, **`occupation`**, **`relationship`**, **`race`**, **`sex`**, and **`native-country`**

## Unique Values
We'll now investigate each columns' unique values.

In [7]:
for c in df.columns:
  print(c + ":", df[c].sort_values().unique(), "\n")

age: [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88
 89 90] 

workclass: ['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay' nan] 

fnlwgt: [  12285   13492   13769 ... 1455435 1484705 1490400] 

education: ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college'] 

education-num: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] 

marital-status: ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed'] 

occupation: ['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-s

From the code output above, we can see various notable aspects of our dataset:

* **`workclass`**, **`occupation`**, and **`native-country`** are the only columns with **`null`** values
* **`workclass`**, **`occupation`**, and **`native-country`** are the only columns with missing values (which are represented by a **`?`** character
* all of our columns have more than one unique value which means all of our columns have variance
* **`income`** is neither binary nor numerical
* many of the columns are categorical

When we are data cleaning, we will have to take the following steps:
* replace the missing values with either **`null`** values or fill them with an appropriate value (e.g. with **`mode`** imputation)
* decide if we'll drop the **`null`** values or replace them with an appropriate value (e.g. with **`mode`** imputation)
* convert **`income`** into a boolean variable
* encode categorical variables

# Data Cleaning
Data cleaning is an important step in the data preprocessing stage because it ensures our data is high-quality and our information is reliable.

## Features with Zero Variance
We'll remove any features with **only one unique value** because they will have zero variance; a feature with zero variance is uninformative in predicting our target.

From the above exploratory analysis, we found that all of our columns have variance; however, we should explicitly check for it

In [8]:
# check the count of unique values in columns
featureValues = {}
for c in df.columns.tolist():
  count = df[c].nunique()
  if count == 1:
    featureValues[c] = count
# list columns with only one unique value
print("Columns with only one unique value:", list(featureValues.keys()))

Columns with only one unique value: []


From the code output above, we can see our dataset has no features with zero variance.

## Drop Columns
We'll drop **`fnlwgt`** and **`education`**.

According to the [U.S. Census Bureau](https://www.census.gov/programs-surveys/cps/technical-documentation/methodology/weighting.html), **`fnlwgt`** (*final weight*) is a "rough measure of the number of actual persons that the sample person represents"; thus, **`fnlwgt`** won't be helpful in predicting our target as it is neither a personal or occupational factor.

**`education-num`** is **`education`** ordinally-encoded; thus, **`education-num`** and **`education`** represent the same information and one of them must be dropped to avoid multicollinearity. We'll drop **`education`** because we'll need to encode our categorical variables later on; keeping **`education-num`** instead of **`education`** will save us the trouble of encoding **`education`** in the future.

In [9]:
for i in range(1, df['education'].nunique() + 1):
  print(df['education'][df['education-num'] == i].unique())

['Preschool']
['1st-4th']
['5th-6th']
['7th-8th']
['9th']
['10th']
['11th']
['12th']
['HS-grad']
['Some-college']
['Assoc-voc']
['Assoc-acdm']
['Bachelors']
['Masters']
['Prof-school']
['Doctorate']


From the code output above, each ordinal value in **`education-num`** is encodes a unique value in **`education`**; thus, **`education`** is correctly encoded by **`education-num`** and **`education`** can be dropped.

In [10]:
df.drop(['fnlwgt', 'education'], inplace=True, axis=1)
df.rename(columns={'education-num':'education'}, inplace=True)

## Drop Null Rows
**`null`** values as they will negatively impact our model so we must decide to drop the **`null`** values or replace them with an appropriate value (e.g. with **`mode`** imputation).

In [11]:
# check null rows
df.isnull().sum()

age                 0
workclass         963
education           0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64

In [12]:
# replace missing values, '?', with null values
df.replace('?', None, inplace=True)
# check null rows
df.isnull().sum()

age                  0
workclass         2799
education            0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
income               0
dtype: int64

In [13]:
# drop null rows
df.dropna(inplace=True)
# shape of dataset
print("Shape of dataframe is:", df.shape)

Shape of dataframe is: (45222, 13)


Dropping the rows with **`null`** values reduces our dataset by about **`7%`** (from **`48842`** rows to **`45222`** rows) which still leaves us with enough observations to construct our model.

## Binary Target Variable

For our classication model, our target variable must be a boolean variable. Currently, **`income`** is neither; **`income`** contains the following values: **`<=50K`**, **`<=50K.`**, **`>50K`**, and **`>50K.`**.

We'll convert **`income`** into a binary variable.

In [14]:
target = 'income'
# make 'income' binary
df[target].replace('<=50K.', '<=50K', inplace=True)
df[target].replace('>50K.', '>50K', inplace=True)

# Exploratory Data Analysis

# Feature Engineering
For a classification model, feature engineering is important to transforming our variables into a format suitable for supervised learning.

In [15]:
# split into features and target
y = df.pop('income')
X = df

## Boolean Target Variable

For our classication model, our target variable must be a boolean variable. Currently, **`income`** is binary but not boolean; **`income`** contains the following values: **`<=50K`** and **`>50K`**.

We'll encode **`income`** with **`0`** for the majority class and **`1`** for the minority class.

In [16]:
# check class percentage
index = y.value_counts().index
count = y.value_counts()
for i in range(2):
  percentage = count[i] / len(df) * 100
  print("class='%s', count=%d, percentage=%.3f%%" % (index[i], count[i], percentage))
# encode 'income'
y = LabelEncoder().fit_transform(y)

class='<=50K', count=34014, percentage=75.216%
class='>50K', count=11208, percentage=24.784%


Our dataset has a class imbalance; the majority class **`<=50K`** is three times larger than the minority class **`>50K`**. Class imbalances cause our model to bias towards the majority class because there is not enough data to learn about minority class patterns.

We'll use class weighting to handle the class imbalance.

## Categorical Variables
For our classification model, our categorical variables must be encoded; the logistic regression algorithm won't work if our categorical variables are not encoded as numerical values.

We'll use dummy-encoding to encode our categorical variables.

In [17]:
X['sex'] = X['sex'].apply(lambda x: 0 if x=='Male' else 1)
# list of categorical variables
cat_features = X.select_dtypes(['object']).columns
# create dummy variables
dummies = pd.get_dummies(X[cat_features], drop_first=True, dtype=int)
# concatenate dummies to orginal dataframe
X = pd.concat([X, dummies], axis=1)
# drop original columns
X.drop(cat_features, axis=1, inplace=True)

# Train-Test Split
We'll divide our dataset into two subsets: **`train`** and **`test`**. Our **`train`** subset will be used to construct our model. Our **`test`** subset will be used to evaluate our model.

In [18]:
# split into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
# shape of train and test data
print("train size X : ", X_train.shape)
print("train size y : ", y_train.shape)
print("test size X : ", X_test.shape)
print("test size y : ", y_test.shape)

train size X :  (31655, 80)
train size y :  (31655,)
test size X :  (13567, 80)
test size y :  (13567,)


After splitting our dataset, we have **`31655`** observations in the **`train`** subset and **`13567`** observations in the **`test`** subset.

# Feature Scaling
For our classification model, our numerical variables should all be scaled to the same range. The algorithm used to construct our Logistric Regression assumes variables which vary greatly in magnitude are more relevant than variables with a smaller magnitude; thus, our model will be biased towards those variables.

In [19]:
# scale features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression Model
We'll construct our initial logistic regression model using default paramenters and improve it with hyperparameter tuning.

As mentioned above, we'll using class-weighting to handle our class imbalance.

In [20]:
# build logistic regression with class-weighting and default parameters
lr1 = LogisticRegression(class_weight={0:0.25, 1:0.75})
# train model
lr1.fit(X_train, y_train)
# make predictions on test subset
y_pred = lr1.predict(X_test)
# confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred), "\n")
# f1 score
print("F1-score:", f1_score(y_test, y_pred))

Confusion Matrix: 
 [[8053 2140]
 [ 538 2836]] 

F1-score: 0.6792814371257484


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Hyperparameter Tuning
While the model learns parameters from the data, we can dictate hyperparameters to shape the model's structure. We'll use **`GridSearch`** to find the most optimal hyperparameters.

In [21]:
# initialize instance of logistic regression
lr = LogisticRegression()
# set range for class-weighting
weights = np.linspace(0.0, 0.99, 10)
# specify hyperparameters
parameters = {'C':[0.1, 0.5, 1, 10, 15, 20], 'penalty':['l1', 'l2'], 'class_weight':[{0:x, 1:1.0-x} for x in weights]}
# create 5 folds
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# GridSearch for hyperparameter tuning
model = GridSearchCV(estimator=lr, param_grid=parameters, scoring='f1', cv=folds, return_train_score=True)
# train model
model.fit(X_train, y_train)
# print best hyperparameters
print("F1 score:", model.best_score_)
print("Hyperparameters:", model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

F1 score: 0.6919382914754892
Hyperparameters: {'C': 15, 'class_weight': {0: 0.33, 1: 0.6699999999999999}, 'penalty': 'l2'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Tuned Logistic Regression
Now we'll construct our model using the optimal parameters we found.

In [22]:
# construct logistic regression with optimal hyperparameters
lr2 = LogisticRegression(C=15, class_weight={0:33, 1:0.6699999999999999}, penalty='l2')
lr2.fit(X_train, y_train)
# predict probabilities on test subset and take probability for class 1([:1])
y_pred_prob = lr2.predict_proba(X_test)[:, 1]
# predict target on test subset
y_pred = lr2.predict(X_test)
# confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred), "\n")
# ROC-AUC score
print("ROC-AUC score:", roc_auc_score(y_test, y_pred_prob))
# precision score
print("Precision socre:", precision_score(y_test, y_pred))
# recall score
print("Recall score:", recall_score(y_test, y_pred))
# f1 score
print("F1 score:", f1_score(y_test, y_pred))

Confusion Matrix: 
 [[10192     1]
 [ 3142   232]] 

ROC-AUC score: 0.9053882765646147
Precision socre: 0.9957081545064378
Recall score: 0.06876111440426794
F1 score: 0.1286387579706127


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Decision Tree Model
Decision tree algorithms are commonly used for supervised, classificiation tasks. While logistic regression adheres to certain assumptions, we don't need to follow any assumptions for decision trees.

In [23]:
# build decision tree with basic parameters
clf1 = DecisionTreeClassifier(random_state=42)
clf1.fit(X_train, y_train)
# accuracy for train subset and test subset
clf1.score(X_train, y_train), clf1.score(X_test, y_test)

(0.9743168535776339, 0.8160241763101643)

## Hyperparameter Tuning
We'll now search for the most optimal hyperparameters for our decision tree using **`GridSearch`**.

In [24]:
# instantiate instance of decision tree
clf = DecisionTreeClassifier()
# specifiy hyperparameters
param_grid = {'criterion': ['entropy','gini'],
              'max_depth': list(range(1, 10)),
              'min_samples_split': list(range(2, 10)),
              'min_samples_leaf': list(range(1, 10))}
# GridSearch for hyperparameter tuning
model = GridSearchCV(clf, param_grid, cv=5)
model.fit(X_train, y_train)
# print best hyperparameters
print("Hyperparameters:", model.best_params_)
print("Score:", model.best_score_)
print("Accuracy:", model.best_estimator_.score(X_test, y_test))

Hyperparameters: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 2}
Score: 0.8530090033170115
Accuracy: 0.8575219282081521


## Tuned Decision Tree
Now we'll construct our model using the optimal parameters we found.

In [28]:
# construct decision tree with optimal hyperparameters
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_leaf=3, min_samples_split=2)
# train decision tree
clf2 = clf2.fit(X_train,y_train)
# predict target on test subset
y_pred = clf2.predict(X_test)
# confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred), "\n")
# ROC-AUC score
print("ROC-AUC score:", roc_auc_score(y_test, y_pred_prob))
# accuracy score
print("Accuracy Score:", accuracy_score(y_test, y_pred))
# precision score
print("Precision socre:", precision_score(y_test, y_pred))
# recall score
print("Recall score:", recall_score(y_test, y_pred))
# f1 score
print("F1 score:", f1_score(y_test, y_pred))

Confusion Matrix: 
 [[9643  550]
 [1373 2001]] 

ROC-AUC score: 0.9053882765646147
Accuracy Score: 0.8582590108351146
Precision socre: 0.7843982751862015
Recall score: 0.5930646117368109
F1 score: 0.6754430379746836
